In [2]:
import sys 
import os 
import time 
import re
import datetime
import pandas as pd 
import numpy as np 
from collections import Counter
import pandas_profiling

# 读取处理好的数据并进行变量衍生

In [3]:
file_path_out = '/Users/yantingting/Documents/海外风控/印尼/MGI数据测试/my_analysis/'

In [12]:
my_data = pd.read_csv(file_path_out + 'my_data1.csv')

/Users/yantingting/anaconda3/envs/modeling/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
app_dict = {'Tunai Cepat': 'Tunai Cepat',
         'Dana Tercepat': 'Dana Tercepat',
         'Sumber Dana': 'Sumber Dana',
         'Pinjam Uang': 'Pinjam Uang',
         'Wall In': 'Wall In',
         'Dana Segera': 'Dana Segera',
         '': 'other',
         'KoinMu': 'other',
         'Pasar Uang': 'other',
         'One Hope': 'other',
         'Modal Pinjaman': 'other',
         'Lahan Uang': 'other'}
my_data['app_name1'] = my_data['app_name'].apply(lambda x: app_dict.get(x))
my_data['amount'] = my_data['amount'].apply(lambda x: np.nan if isinstance(x,float) else re.search('\d+', x).group())
my_data['amount'] = my_data['amount'].astype('float64')
my_data['cur_fund'] = my_data.apply(lambda x :(datetime.datetime.strptime('2019-12-18', '%Y-%m-%d') - datetime.datetime.strptime(x['Date/Time'], '%Y-%m-%d')).days if x['message'] == '资金已到账' else np.nan, axis = 1)

In [14]:
my_data['loan_suc'] = my_data['message'].apply(lambda x: 1 if x == '资金已到账' else 0)
my_data['pay_suc'] = my_data['message'].apply(lambda x: 1 if x == '还款成功' else 0)
my_data['remind_due'] = my_data['message'].apply(lambda x: 1 if x == '账单已逾期N天请还款' else 0)
my_data['remind_due_sb'] = my_data['message'].apply(lambda x: 1 if x == '提醒某人还款' else 0)
my_data['yzm'] = my_data['message'].apply(lambda x: 1 if x == '验证码' else 0)

In [15]:
frame1 = my_data.groupby(['Send to'])
my_frame = pd.DataFrame()
my_frame['cnt_message'] = frame1['Message ID'].agg('count')
my_frame['cnt_unique_app'] = frame1.agg({'app_name':pd.Series.nunique})
my_frame['cnt_loan_suc'] = frame1['loan_suc'].agg('sum')
my_frame['cnt_pay_suc'] = frame1['pay_suc'].agg('sum')
my_frame['cnt_remind_due'] = frame1['remind_due'].agg('sum')
my_frame['cnt_remind_due_sb'] = frame1['remind_due_sb'].agg('sum')
my_frame['cnt_yzm'] = frame1['yzm'].agg('sum')
my_frame['min_cur_fund'] = frame1['cur_fund'].agg('min')
my_frame['max_cur_fund'] = frame1['cur_fund'].agg('max')
my_frame['min_amount'] = frame1['amount'].agg('min')
my_frame['max_amount'] = frame1['amount'].agg('max')
my_frame.to_csv(file_path_out + 'MGI_result_data.csv')

In [16]:
my_frame = pd.read_csv(file_path_out + 'MGI_result_data.csv')
my_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310970 entries, 0 to 310969
Data columns (total 12 columns):
Send to              310970 non-null int64
cnt_message          310970 non-null int64
cnt_unique_app       310970 non-null int64
cnt_loan_suc         310970 non-null int64
cnt_pay_suc          310970 non-null int64
cnt_remind_due       310970 non-null int64
cnt_remind_due_sb    310970 non-null int64
cnt_yzm              310970 non-null int64
min_cur_fund         56985 non-null float64
max_cur_fund         56985 non-null float64
min_amount           98716 non-null float64
max_amount           98716 non-null float64
dtypes: float64(4), int64(8)
memory usage: 28.5 MB


In [17]:
my_frame['cnt_gap'] = my_frame['cnt_loan_suc'] - my_frame['cnt_pay_suc']
my_frame['成功放款_flag'] = my_frame['cnt_loan_suc'].apply(lambda x: 1 if x>0 else 0)
my_frame['验证码_flag'] = my_frame['cnt_yzm'].apply(lambda x: 1 if x>0 else 0)

In [18]:
type(my_frame['验证码_flag'][19])

numpy.int64

In [21]:
df1 = my_frame[(my_frame['成功放款_flag'] == 1)|(my_frame['验证码_flag'] == 1)]
df1.info()
df1.to_csv(file_path_out + 'df_var_1227.csv', index = False)
df1[['Send to']].to_csv(file_path_out + 'phone1227.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176754 entries, 0 to 310969
Data columns (total 15 columns):
Send to              176754 non-null int64
cnt_message          176754 non-null int64
cnt_unique_app       176754 non-null int64
cnt_loan_suc         176754 non-null int64
cnt_pay_suc          176754 non-null int64
cnt_remind_due       176754 non-null int64
cnt_remind_due_sb    176754 non-null int64
cnt_yzm              176754 non-null int64
min_cur_fund         56985 non-null float64
max_cur_fund         56985 non-null float64
min_amount           63402 non-null float64
max_amount           63402 non-null float64
cnt_gap              176754 non-null int64
成功放款_flag            176754 non-null int64
验证码_flag             176754 non-null int64
dtypes: float64(4), int64(11)
memory usage: 21.6 MB


# 号码导入数据库

In [2]:
import sys
import psycopg2
import pandas as pd
from collections import Counter
from jinja2 import Template

In [3]:
def upload_df_to_pg(sql):
    """
    If sql is table creation, then rows = [[True]] is succeed.
    If sql is insert query, then rows = [[4]] where 4 is changeable according to
    number of rows inserted
    """
    usename = "postgres"
    password = "Mintq2019"
    db = "risk_dm"
    host = "192.168.2.19"
    port = "5432"
    try:
        conn = psycopg2.connect(database=db, user=usename, password=password, host=host, port=port)
        # print("Opened database successfully")
    except Exception as e:
        print(e)
    cur = conn.cursor()
    cur.execute(sql)
    try:
        conn.commit()
        return 'success'
    except:
        print("upload failed, check query")
        return 'failure'

In [6]:
my_phone_data = pd.read_csv(file_path_out + 'phone1227.csv',dtype={'Send to':'str'})
my_phone_data.rename(columns = {'Send to': 'phone', 'Unnamed: 0': 'index'}, inplace = True)
SQL_CREATE_TABLE = """
CREATE TABLE temp1227_phone(
    index int,
    phone varchar
)
"""
upload_df_to_pg(SQL_CREATE_TABLE)

insert = """
INSERT INTO temp1227_phone
   VALUES
 {% for var in var_list %}
{{ var }},
{% endfor %}
"""

var_list = []
upload_var_order = ['']
for cols, rows in my_phone_data.iterrows():    
    c = tuple(rows)
    #print(c)
    var_list.append(c)

insert_sql = Template(insert).render(var_list=var_list)[:-2]
insert_sql = insert_sql.replace('\n\n','')
upload_df_to_pg(insert_sql)

'success'

'success'

# 匹配符合条件的号码发送短信

In [7]:
# 从来没有在uku有过成功订单的用户
query1 = '''
select phone, loan_no, effective_date, cell_phone
from  public.temp1227_phone t1
left join 
(
select a1.*, a2.cell_phone
from
(select * 
from  (select loan_no, customer_id,effective_date,row_number()over(partition by customer_id order by effective_date desc) as loan_num
from dw_gocash_go_cash_loan_gocash_core_loan a1 ) t 
where loan_num = 1 ) a1 
left join dw_gocash_go_cash_loan_gocash_core_customer a2 
on a1.customer_id = a2.id)t2 
on substring(t1.phone,3) = substring(t2.cell_phone,2)
'''

In [9]:
df2 = data_con.database_gn(query1)
df2.info()
df2.to_csv(file_path_out + 'df_phone1227.csv', index = False)

Successful connection
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176754 entries, 0 to 176753
Data columns (total 4 columns):
phone             176754 non-null object
loan_no           76457 non-null object
effective_date    76457 non-null object
cell_phone        76457 non-null object
dtypes: object(4)
memory usage: 5.4+ MB


In [8]:
df1 = pd.read_csv(file_path_out + 'df_var_1227.csv')
df2 = pd.read_csv(file_path_out + 'df_phone1227.csv')
df2['is_fund'] = df2['loan_no'].apply(lambda x: 0 if x is np.nan else 1)
# df_all = pd.merge(df1[['Send to', '成功放款_flag', '验证码_flag']], df2[['phone', 'is_fund']],left_on='Send to', right_on='phone', how='left')
# df_all[['phone', 'is_fund', '成功放款_flag', '验证码_flag']].to_csv(file_path_out + 'test_phone1227.csv')

In [9]:
df1.head()

,Send to,cnt_message,cnt_unique_app,cnt_loan_suc,cnt_pay_suc,cnt_remind_due,cnt_remind_due_sb,cnt_yzm,min_cur_fund,max_cur_fund,min_amount,max_amount,cnt_gap,成功放款_flag,验证码_flag
0,6280000000,3,1,0,0,0,0,3,NaN,NaN,NaN,NaN,0,0,1
1,6281111111,1,1,0,0,0,0,1,NaN,NaN,NaN,NaN,0,0,1
2,6281111222,1,1,0,0,0,0,1,NaN,NaN,NaN,NaN,0,0,1
3,6281222620,1,1,0,0,0,0,1,NaN,NaN,NaN,NaN,0,0,1
4,6281225709,11,1,4,4,0,0,0,22.0,41.0,1008000.0,1400000.0,0,1,0


In [26]:
df_all = pd.merge(df1[['Send to', '成功放款_flag', '验证码_flag','cnt_remind_due']], df2[['phone', 'is_fund']],left_on='Send to', right_on='phone', how='left')
df_all['有逾期短信_flag'] = df_all['cnt_remind_due'].apply(lambda x: 1 if x>0 else 0 )
df_all[['phone', 'is_fund', '成功放款_flag', '验证码_flag','有逾期短信_flag']].to_excel(file_path_out + 'MGI测试号码1227.xlsx', index = False)

In [21]:
Counter(df_all['temp'])

Counter({1: 84445, 0: 92309})